In [1]:
import pandas as pd
%matplotlib inline

In [2]:
path = r"C:\Users\anees\Documents\CS\MP\datasets\2000c_200sub.csv"

In [3]:
df = pd.read_csv(path)

In [4]:
df.head()

,subreddit,user,comment,score
0,funny,papa_mike2,At least check the mail while you’re down ther...,10749.0
1,funny,notagoodboye,He missed cracking his head open on that littl...,14585.0
2,funny,Rati0h,I’ve never seen shoes leave a mans body as vio...,7169.0
3,funny,whezel,“I didn’t fuckin do it on purpose” is the best...,16282.0
4,funny,MB8189,"The audio is perfect, “didn’t fucking do it on...",2380.0


In [5]:
from sklearn.externals import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
import pickle

In [6]:
classifiers = []
for i in range(6):
    classifiers.append(joblib.load("lr_char_" + str(i) + ".joblib"))

In [7]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 6),
    max_features=50000)

In [8]:
word_vectorizer = pickle.load(open("word_vectorizer.pickle",'rb'))
char_vectorizer = pickle.load(open("char_vectorizer.pickle",'rb'))

In [12]:
df = df.dropna()

In [14]:
print("generating word features")
df_wordfeat = word_vectorizer.transform(df.comment)
print("creating char features")
df_charfeat = char_vectorizer.transform(df.comment)
print("done")
df_feat = hstack([df_charfeat, df_wordfeat])

generating word features
creating char features
done


In [15]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [16]:
df['toxic'] = 0
df['severe_toxic'] = 0
df['obscene'] = 0
df['threat'] = 0
df['insult'] = 0
df['identity_hate'] = 0

In [17]:
i = 0
for class_name in class_names:
    df[class_name] = classifiers[i].predict_proba(df_feat)[:,1]
    i+=1

In [18]:
df.describe()

,score,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,390586.000000,390586.000000,390586.000000,390586.000000,390586.000000,390586.000000,390586.000000
mean,81.013418,0.125756,0.009367,0.064549,0.003518,0.050982,0.009867
std,736.857267,0.156182,0.024595,0.130917,0.003763,0.082617,0.011639
min,-1204.000000,0.002332,0.000868,0.002535,0.000851,0.002098,0.001703
25%,1.000000,0.050795,0.004893,0.023822,0.002601,0.022348,0.006725
50%,4.000000,0.076973,0.006297,0.032371,0.003059,0.031116,0.008339
75%,16.000000,0.124978,0.008201,0.046723,0.003630,0.046098,0.010387
max,84143.000000,0.999986,0.885255,0.999992,0.340690,0.999421,0.815637


In [19]:
df.to_csv("toxic200.csv", index=False)